In [ ]:
"""
Set up yagmail: https://towardsdatascience.com/automate-sending-emails-with-gmail-in-python-449cc0c3c317

Set up auto schedule: https://towardsdatascience.com/10-step-guide-to-schedule-your-script-using-cloud-services-7f4a8e517d81

Enjoy!
"""

!pip3 install yagmail
 
from urllib.request import urlopen
import yagmail

user = 'your_id'
app_password = 'generate_your_app_passwrd'
to = 'sender\'s_address'

try:
    def jobNotiSSC(i):
        dateList = []
        contentList = []
        urlList = []
        data = ""
        j = 1
        url = "https://ssc.nic.in/"

        page = urlopen(url)
        html_bytes = page.read()
        html = html_bytes.decode("utf-8")

        firstIndex = html.find("<div class=\"leftsideNotifications_New\">")
        firstIndex = firstIndex + \
            len("<div class=\"leftsideNotifications_New\">")

        while i > 0:

            firstIndex = html.find(
                "<div class=\"eachNotification \">", firstIndex)
            firstIndex = firstIndex+len("<div class=\"eachNotification \">")
            firstIndex = html.find("<span>", firstIndex)
            firstIndex = firstIndex+len("<span>")
            lastIndex = html.find("</span>", firstIndex)

            # Date is fetched here
            date = html[firstIndex:lastIndex]
            date = date.split("<i>")
            date = " ".join(date)
            date = date.split("</i>")
            date = " ".join(date)

            # Linked document URL is fetched here.
            firstIndex = html.find("<a href=", lastIndex)
            firstIndex = firstIndex+len("<a href=\"")
            lastIndex = html.find("target=\"_blank\">", firstIndex)
            urlDoc = html[firstIndex:lastIndex].strip()
            urlDoc = urlDoc.replace(" ", "%20")
            urlDoc = urlDoc.replace("\"", "")

            # Content will be fetched here.
            firstIndex = lastIndex+len("target=\"_blank\">")
            lastIndex = html.find("</a>", firstIndex)
            content = html[firstIndex:lastIndex].strip()

            dateList.append(date)
            contentList.append(content)
            urlList.append(urlDoc)
            j = j+1
            i = i-1

        return dateList, urlList, contentList

    def html_builder():
        header = """\
<!DOCTYPE html PUBLIC "-//W3C//DTD XHTML 1.0 Transitional //EN" "http://www.w3.org/TR/xhtml1/DTD/xhtml1-transitional.dtd">
<html>
  <head>
    <meta http-equiv="Content-Type" content="text/html; charset=UTF-8" />
    <meta name="viewport" content="width=device-width, initial-scale=1.0" />
    <meta name="x-apple-disable-message-reformatting" />
    <!--[if !mso]><!-->
    <meta http-equiv="X-UA-Compatible" content="IE=edge" />
    <!--<![endif]-->     
  </head>
  <body>
    <table>
      <tr>
        <th>Sr. No.</th>
        <th>Date</th>
        <th>Link</th>
        <th>Content</th>
      </tr>"""

        body = '''<tr>
        <td style="padding:20px"><?a?></td>
        <td style="padding:20px"><?b?></td>
        <td style="padding:20px"><a href="<?c?>" target="_blank">Link</a></td>
        <td style="padding:20px"><b><?d?></b></td>
      </tr>'''

        footer = """</table>
        <p><a href="https://ssc.nic.in/">SSC Online</a></p>
  </body>
</html>"""

        new_str = ''

        for i in range(no_of_data):
            temp = body
            str1 = "dateList["+str(i)+"]"
            str2 = "urlList["+str(i)+"]"
            str3 = "contentList["+str(i)+"]"

            temp = temp.replace("<?a?>", str(i+1))
            temp = temp.replace("<?b?>", str1)
            temp = temp.replace("<?c?>", str2)
            temp = temp.replace("<?d?>", str3)
            new_str += temp

        content = header + new_str + footer

        for i in range(no_of_data):
            str1 = "dateList["+str(i)+"]"
            str2 = "urlList["+str(i)+"]"
            str3 = "contentList["+str(i)+"]"

            content = content.replace(str1, str(dateList[i]))
            content = content.replace(str2, str(urlList[i]))
            content = content.replace(str3, str(contentList[i]))

        return content

    """main"""
    no_of_data = 3

    dateList, urlList, contentList = jobNotiSSC(no_of_data)

    subject = 'Job Alert'

    content = html_builder()
    # print(content)
    with yagmail.SMTP(user, app_password) as yag:
        yag.send(to, subject, content)
        print('Sent email successfully')
except:
    print("Error occured")